# Stage C Lesson 1 Linear Classification and Logistics Regression

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
Url = 'https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae.csv'
df=pd.read_csv(Url)
df.head()

C:\Users\solas\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [3]:
#To count and know the distribution of the target variable
#target variable is QScore

df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1B       16
1A       16
Name: QScore, dtype: int64

In [4]:
#to confirm my pandas version
#not relevant to this work kindly ignore 

pd.__version__

'1.1.0'

In [5]:
#to proceed with cleaning we will first identify the rows with missing values 

df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [6]:
#to simplify our work,
#we will drop the rows with missing values

df = df.dropna()

In [7]:
#to confirm the missing rows have been dropped 

df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [8]:
#considering the effect of the drop
#we eill check the count on our target variable 

df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [9]:
#our result indicate we have three classes left from five
#the count also reveled the imbalance in the classes
#making use of binary classification, we will cmbine class 2A and 1A

df['QScore']= df['QScore'].replace(['1A'], '2A')


In [10]:
df['QScore'].value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [11]:
#storing class 2A in a variable 

df_2A = df[df['QScore'] == '2A']


In [12]:
#i think considering the size of class 3A, we will be making random selection
#randomly selecting 350 rows for the 3A variable
#A random_state parameter is added for reproducibility

df_3A = df[df['QScore'] == '3A'].sample(350, random_state=0)


In [13]:
#joining the two data frame together
#making use of the append function

data_df = df_2A.append(df_3A)


In [14]:
#viualizing our new dataframe

data_df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
1536,Algeria,2016,4,AreaPerCap,2.072989e-01,8.112722e-01,0.048357265,0.022585,2.998367e-02,0.000000,1.119497e+00,2A
1537,Algeria,2016,4,AreaTotHA,8.417600e+06,3.294260e+07,1963600,917100.000000,1.217520e+06,0.000000,4.545842e+07,2A
1538,Algeria,2016,4,BiocapPerCap,2.021916e-01,2.636077e-01,0.027166736,0.007948,2.924496e-02,0.000000,5.301590e-01,2A
1539,Algeria,2016,4,BiocapTotGHA,8.210214e+06,1.070408e+07,1103135.245,322736.916200,1.187524e+06,0.000000,2.152769e+07,2A
1540,Algeria,2016,4,EFConsPerCap,6.280528e-01,1.810332e-01,0.162800822,0.014729,2.924496e-02,1.391455,2.407316e+00,2A


In [15]:
data_df.tail()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
16241,Denmark,1999,54,AreaTotHA,3.257548e+06,1.331344e+05,469158.7,9.962839e+06,3.140515e+05,0.000000e+00,1.413673e+07,3A
38504,Malaysia,1972,131,AreaPerCap,3.286831e-01,3.160671e-02,1.878321615,3.430040e+00,2.527423e-02,0.000000e+00,5.693926e+00,3A
63831,Turkey,2016,223,EFConsTotGHA,6.519846e+07,9.150603e+06,26216874.18,3.101630e+06,2.836436e+06,1.604640e+08,2.669680e+08,3A
25784,Guinea,1982,90,AreaPerCap,7.404170e-01,2.281158e+00,1.549554248,1.028909e+00,3.689592e-02,0.000000e+00,5.636934e+00,3A
10836,Central African Republic,1993,37,EFConsPerCap,3.007898e-01,7.340857e-01,0.486456419,7.416127e-03,2.283253e-02,4.200042e-02,1.593581e+00,3A


In [16]:
#to confirm the size of our new dataset

data_df.shape

(590, 12)

In [17]:
#counting the value of our target varaible (considering the new dataset)  

data_df.QScore.value_counts()


3A    350
2A    240
Name: QScore, dtype: int64

In [18]:
#we need to shuffle the data since some changes has occure on the dataset
#shuffling also ensure the data point creates an independent change

import sklearn.utils

data_df = sklearn.utils.shuffle(data_df, random_state = 0)

#it is always important to set it random state 

In [19]:
#we will also reset the index and drop the current one
#for a more orderly state for the new dataset

data_df = data_df.reset_index(drop = True)


In [20]:
#confirming the size

data_df.shape

(590, 12)

In [21]:
#visualizing the effected change 

data_df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Trinidad and Tobago,2016,220,AreaTotHA,4.700000e+04,7.000000e+03,236100,2.113600e+06,1.291780e+03,0.000000e+00,2.404992e+06,2A
1,Belize,2016,23,EFProdPerCap,5.071023e-01,2.055006e-01,0.20984546,2.222697e+00,5.540510e-04,4.805086e-01,3.626208e+00,2A
2,Guatemala,2016,89,EFConsTotGHA,7.289958e+06,2.665859e+06,9354954.762,5.894037e+05,1.496289e+06,9.760726e+06,3.115719e+07,2A
3,Somalia,1989,201,BiocapPerCap,1.477893e-01,7.990278e-01,0.370905175,4.726897e-01,5.685511e-02,0.000000e+00,1.847267e+00,3A
4,Kyrgyzstan,2016,113,EFConsPerCap,4.271284e-01,1.977978e-01,0.059306959,5.479879e-03,7.480447e-02,8.907022e-01,1.655220e+00,2A


In [22]:
#for the purpose of this course we will be dropping some coloumns that wont be needed
#the columns include; country code, country and year 

data_df = data_df.drop(['country_code', 'country', 'year'], axis = 1)

In [23]:
data_df.head()

,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,AreaTotHA,4.700000e+04,7.000000e+03,236100,2.113600e+06,1.291780e+03,0.000000e+00,2.404992e+06,2A
1,EFProdPerCap,5.071023e-01,2.055006e-01,0.20984546,2.222697e+00,5.540510e-04,4.805086e-01,3.626208e+00,2A
2,EFConsTotGHA,7.289958e+06,2.665859e+06,9354954.762,5.894037e+05,1.496289e+06,9.760726e+06,3.115719e+07,2A
3,BiocapPerCap,1.477893e-01,7.990278e-01,0.370905175,4.726897e-01,5.685511e-02,0.000000e+00,1.847267e+00,3A
4,EFConsPerCap,4.271284e-01,1.977978e-01,0.059306959,5.479879e-03,7.480447e-02,8.907022e-01,1.655220e+00,2A


In [24]:
#assigning our features into different variables 
#we will be dropping our target variable when assigning the input ie. X 
#because we dont want to put the target feature among the input feature 

X = data_df.drop(columns = 'QScore', axis =1 )
Y = data_df['QScore']

#when x is the input 
#and y is the output or target

In [25]:
#we then split the data into traning and testing sets 

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state =0)
Y_train.value_counts()

3A    248
2A    165
Name: QScore, dtype: int64

In [26]:
#from the dataset some of the column comes in object as against float,
#to convert to numerical we make use of encoder label
#we will get the visual of our input

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413 entries, 285 to 559
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   record          413 non-null    object 
 1   crop_land       413 non-null    float64
 2   grazing_land    413 non-null    float64
 3   forest_land     413 non-null    object 
 4   fishing_ground  413 non-null    float64
 5   built_up_land   413 non-null    float64
 6   carbon          413 non-null    float64
 7   total           413 non-null    float64
dtypes: float64(6), object(2)
memory usage: 29.0+ KB


In [27]:
#to encode the variable using Label encoder

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
X_train.record = encoder.fit_transform(X_train.record)
X_test.record = encoder.transform(X_test.record)

C:\Users\solas\anaconda3\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [28]:
#to confirm the changes have been effected 

X_train.head()

,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total
285,5,9.979776e+06,443742.397300,8.22956e+06,9.431555e+05,1.203837e+06,3.713204e+06,2.451327e+07
113,6,1.577776e-01,0.020945,0.767404124,1.954311e-02,3.713408e-02,6.699166e-02,1.069796e+00
18,4,4.461706e-01,0.237536,0.423942833,1.122089e-01,4.755381e-02,8.308703e-01,2.098283e+00
76,7,3.390146e+07,476731.570100,7628142.139,1.926862e+07,5.888916e+06,3.415330e+07,1.013172e+08
206,4,3.407583e-01,0.008844,0.150235593,1.281046e-02,2.797109e-02,1.704237e-01,7.110435e-01


In [29]:
#noting the imbalance in the class distribution of the output ie. Qscore from line 25
#we will use SMOTE but only on the training data

import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 1)
X_train_balanced, Y_balanced = smote.fit_sample(X_train, Y_train)

In [30]:
#checking the new shape of the input ie. X variable 

X_train.shape, X_train_balanced.shape

((413, 8), (496, 8))

In [31]:
#checking the new shape of the output/target variable ie. Y variable 

Y_train.shape, Y_balanced.shape

((413,), (496,))

In [32]:
#to confirm if we now have a balanced variable/class

pd.Series(Y_balanced.tolist()).value_counts()

3A    248
2A    248
dtype: int64

In [33]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(X_train_balanced.drop(columns =['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns = X_train_balanced.drop(columns = ['record']).columns)
normalised_train_df['record'] = X_train_balanced['record']

In [34]:
X_test = X_test.reset_index(drop=True)
normalised_test_df = scaler.transform(X_test.drop(columns =['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns = X_test.drop(columns = ['record']).columns)
normalised_test_df['record'] = X_test['record']

In [35]:
normalised_train_df.head()

,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,record
0,1.787946e-02,1.143006e-03,1.017073e-02,1.956717e-03,9.671065e-03,2.361315e-03,1.128275e-02,5
1,2.529569e-10,5.395099e-11,9.484182e-10,4.054509e-11,2.983178e-10,4.260159e-11,3.946518e-10,6
2,7.696329e-10,6.118535e-10,5.239418e-10,2.327940e-10,3.820250e-10,5.283701e-10,8.680348e-10,4
3,6.073681e-02,1.227981e-03,9.427456e-03,3.997563e-02,4.730879e-02,2.171889e-02,4.663338e-02,7
4,5.807795e-10,2.278167e-11,1.856729e-10,2.657720e-11,2.247066e-10,1.083765e-10,2.295285e-10,4


In [36]:
#instantiate the logisticregression model
#we also fit the normalised train data with the y_balanced

from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, Y_balanced)

LogisticRegression()

In [37]:
LogisticRegression(C=1.0, class_weight = None, dual = False, fit_intercept = True, intercept_scaling = 1, l1_ratio = None, max_iter = 100, multi_class = 'auto', n_jobs = None, penalty= 'l2', random_state = None, solver = 'lbfgs', tol = 0.0001, verbose=0, warm_start = False)

LogisticRegression()

# 

# Stage C Lesson 2 Measuring Classification Performance

# 1. Cross-Validation and Accuracy

In [38]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, Y_balanced, cv=5, scoring = 'f1_macro')
scores

array([0.49164851, 0.50687856, 0.50179727, 0.57475118, 0.52963801])

In [39]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores = []

In [40]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import recall_score, accuracy_score, precision_score, confusion_matrix, f1_score

for train_index, test_index in kf.split(normalised_train_df):
    X_train, X_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    Y_train, Y_test = Y_balanced[train_index], Y_balanced[test_index]
    model = LogisticRegression().fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    f1_scores.append(f1_score(Y_test, Y_pred, pos_label='2A')*100)
    

In [41]:
f1_scores

[55.639097744360896,
 66.66666666666667,
 47.05882352941176,
 50.79365079365079,
 0.0]

# b. Strafied K-Fold Cross Validation

In [42]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1)
f1_scores_strat = []

In [43]:
for train_index, test_index in skf.split(normalised_train_df, Y_balanced):
    X_train, X_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    Y_train, Y_test = Y_balanced[train_index], Y_balanced[test_index]
    model = LogisticRegression().fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    f1_scores_strat.append(f1_score(Y_test, Y_pred,pos_label='2A'))

In [44]:
np.mean(f1_scores_strat)*100, np.mean(f1_scores)

(62.68507746224905, 44.031647746818024)

# c. Leave One Out Cross Validation (LOOCV)

In [45]:
cross_val_score(LogisticRegression(), normalised_train_df, Y_balanced,cv = skf, scoring='f1_macro').mean()

0.5002303685482283

In [46]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, Y_balanced, cv=loo,scoring= 'f1_macro' )
average_score = scores.mean() * 100

In [47]:
average_score

53.42741935483871

# i. CONFUSION MATRIX

# ii. Accuracy

In [56]:
#Noting from Cross Validation, the values of the y_test was re-assigned,
#to avoid error code,it is important to resplit this because without it the code throws an error
#it is important to note that, it doesn't change values this is the impact of the random_state.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state= 0)

In [57]:
from sklearn.metrics import recall_score, accuracy_score, precision_score,f1_score, confusion_matrix

new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true = Y_test, y_pred = new_predictions, labels = ['2A', '3A'])
cnf_mat

array([[64, 11],
       [76, 26]], dtype=int64)

In [58]:
accuracy = accuracy_score(y_true = Y_test, y_pred = new_predictions)
print('Accuracy: {}'. format(round(accuracy*100),2))

Accuracy: 51


# iii. Precision

In [50]:
precision = precision_score(y_true = Y_test, y_pred = new_predictions, pos_label = '2A' )
print('Precision: {}' .format(round(precision*100), 2))

Precision: 46


# iv. Recall 

In [54]:
recall = recall_score(y_true = Y_test, y_pred = new_predictions, pos_label = '2A')
print('Recall: {}' .format(round(recall*100), 2))

Recall: 85


# v. F1- Score

In [52]:
f1 = f1_score(y_true = Y_test, y_pred = new_predictions, pos_label = '2A')
print('F1: {}' .format(round(f1*100), 2))

F1: 60
